In [41]:
import re
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import numpy as np
import pandas as pd
import lxml
import time
import config

# 加密函数
from pyncm.utils.crypto import WeapiEncrypt

In [42]:
df_songs = pd.read_csv('../data/raw_songs.csv')

In [ ]:
def get_song_info(row):
    original_url = config.SONG_ORG_URL_PRE + row.song_id
    response = requests.get(url=original_url,
                            headers=config.HEADERS,
                            cookies=config.COOKIES,
                            params={})
    soup = bs(response.text, 'lxml')
    song_img_link = soup.find('img', class_='j-img')['data-src']

    # get lyrics, using EncryptFunction from pyncm
    song_lyric_link = config.LYRIC_URL_PRE + config.COOKIES['__csrf']
    data = {
        'id': row.song_id,
        'lv': -1,
        'tv': -1,
        'csrf_token': config.COOKIES['__csrf']
    }
    enc_data = WeapiEncrypt(data)
    lyric_response = requests.post(url=song_lyric_link,
                                   headers=config.HEADERS,
                                   cookies=config.COOKIES,
                                   data=enc_data)
    lyric_soup = bs(lyric_response.txt, 'lxml')
    
    

In [44]:
url = 'https://music.163.com/song?id=1394123837'
response = requests.get(url=url,
                        headers=config.HEADERS,
                        cookies=config.COOKIES,
                        params={})
soup = bs(response.text, 'lxml')


In [45]:
song_img_link = soup.find('img', class_='j-img')['data-src']

In [46]:
# config.COOKIES['__csrf']
song_lyric_link = config.LYRIC_URL_PREFIX + config.COOKIES['__csrf']

In [47]:
# 使用pyncm获取加密参数
data = {
    'id': '1394123837',
    'lv': -1,
    'tv': -1,
    'csrf_token': config.COOKIES['__csrf']
}
enc_data = WeapiEncrypt(data)

In [48]:
lyric_response = requests.post(url=song_lyric_link,
                               headers=config.HEADERS,
                               cookies=config.COOKIES,
                               data=enc_data)

In [ ]:
lyric_soup = bs

'{"sgc":false,"sfy":false,"qfy":false,"lrc":{"version":84,"lyric":"[00:00.000] 作词 : 杨和苏KeyNG\\n[00:01.000] 作曲 : 杨和苏KeyNG/陈令韬\\n[00:02.000] 编曲 : 陈令韬\\n[00:03.000] 制作人 : 陈令韬/杨和苏KeyNG\\n[00:20.820](1949年):\\n[00:22.660]我看着跟前这个漂亮女孩\\n[00:24.410]想起数年前 我鼓起勇气向她表白\\n[00:26.770]现在回想起来 虽然这一路上很辛 苦\\n[00:29.440]但幸 福的是 如今身边 多了几个小孩\\n[00:32.150]我希望今年家里 能有更 多的收 获\\n[00:34.440]让孩子不只是在周末 才能打牙祭\\n[00:36.940]我希望他们都能快点长大\\n[00:38.880]有自 己的想法 最好比我还有文化\\n[00:41.539]我喜欢喝茶 但我最大的兴趣\\n[00:43.803]是每周留个三 毛钱 去看京剧\\n[00:46.300]我喜欢梅兰芳 他是我的男神\\n[00:48.415]在“贵妃醉酒”里的他真的很迷人\\n[00:51.091]我信仰和平 我相信正义\\n[00:53.455]我希望全世界的人民同舟共济\\n[00:55.831]当五星红 旗 在天安门冉冉升起\\n[00:58.355]1949年 是中国人的胜利\\n[01:01.340]你是否还记得 第一次\\n[01:06.629]行走  在“她”的美丽土地上\\n[01:10.549]你是否曾注意过 多少年\\n[01:16.150]之后 “她”还是陪在你左右\\n[01:20.711]Wu~~\\n[01:25.211]“她”爱着你 你也深爱着“她”\\n[01:27.201]“她”给了你第一个温暖的家\\n[01:29.820]Wu~~\\n[01:34.431]和我一起站在“她”的身边\\n[01:36.718]陪着“她”度过下一个70周年\\n[01:39.431]（1979年):\\n[01:39.940]我想穿 上最帅气的西服\\n[01:41.477]跟我心 怡的女孩跳支交谊舞\\n[01:43.760]请你跟上我那飘 逸